# iOS 어플리케이션 구동을 위한 커스텀 음성 인식 모델 훈련

지금부터 간단한 음성 인식을 위한 20kb 정도의 iOS 어플리케이션용 TensorFlow Lite 모델을 훈련하는 방법을 살펴볼것입니다. 이 모델은 micro_speech 예제 어플리케이션에서 사용된 것과 같습니다.

이 모델은 Google Colaboratory 환경에서도 사용할 수 있습니다.

[Run in Google Colab](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train_speech_model_ios_ko.ipynb)	[View source on GitHub](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train_speech_model_ios_ko.ipynb)

이 노트북의 코드는 Python 스크립트를 실행하여 훈련을 진행한 뒤, 모델을 고정시킵니다. 그리고 Tensorflow Lite 모델 변환기(toco)를 통해 iOS 어플리케이션 구동을 위한 가벼운 모델을 생성합니다. 

**GPU 가속을 사용하면 훈련 속도가 훨씬 빨라집니다.** 훈련을 하기에 앞서, **Runtime -> Change runtime type**을 클릭하면 런타임을 **GPU** 환경으로 바꿀 수 있습니다. 훈련에는 GPU환경을 기준으로 18,000 Iteration의 경우 1시간 30분에서 2시간 정도 시간이 소요됩니다.


## 의존성/종속성 설치

최신 버전 텐서플로우와 예제 코드가 작성될 당시 사용했던 텐서플로우의 버전이 다릅니다.
이곳에서는 tesorflow 1.15 버전을 사용하여 훈련을 진행합니다.

In [2]:
# !git clone https://github.com/yunho0130/tensorflow-lite.git 
!git clone https://github.com/sanghunkang/tensorflow-lite.git
!pip install tensorflow==1.15

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


Colab에서 실행중이라면, 왼쪽의 디렉토리에 tensorflow-lite가 추가된 것을 확인할 수 있습니다.

## 커스텀 데이터 생성 및 업로드

훈련에 앞서서 훈련에 사용하고자 하는 데이터를 생성해야 합니다. speech_commands 예제에서는 대략 4,000개의 1초 길이의 16bit wav 파일을 훈련, 테스트 및 추론에 사용하고 있습니다. 클래스(키워드)별로 최소 2,000개 정도의 데이터를 준비하는 것이 좋습니다. 데이터의 개수와 품질은 모델의 성능에 중요한 영향을 줄 수 있습니다. 
 
ffmpeg 도구를 사용하면 변환작업을 빠르고 편리하게 수행할 수 있습니다.


In [ ]:
!for i in *.wav; do ffmpeg -y -i "$i" -acodec pcm_s16le -ac 1 -ar 16000 "tmp/${i%.*}.wav"; done

당신이 만약 window 유저라면 다음 python 코드로 변환작업을 할 수 있습니다.


In [ ]:
import wave
import glob
def pcm2wav( pcm_file, wav_file, channels=1, bit_depth=16, sampling_rate=16000 ):
# Check if the options are valid.
if bit_depth % 8 != 0:
    raise ValueError('bit_depth '+str(bit_depth)+' must be a multiple of 8.')
with open( pcm_file, 'rb') as opened_pcm_file:
    pcm_data = opened_pcm_file.read();
    obj2write = wave.open( wav_file, 'wb')
    obj2write.setnchannels( channels )
    obj2write.setsampwidth( bit_depth // 8 )
    obj2write.setframerate( sampling_rate )
    obj2write.writeframes( pcm_data )
    obj2write.close()
file_list = glob.glob('file_directory/*.wav')
for file in file_list:
   pcm2wav(file, 'new_file_directory/'+file, 1, 16, 16000 )

변환한 파일은 클래스별로 각각 폴더로 만들어서 저장한 후, 이 폴더들을 다시 하나의 디렉토리 안에 모아놓는 것이 좋습니다. 
우리는 이제 이 디렉토리를 데이터 디렉토리 경로로 지정하여 원하는 모델을 훈련할 수 있습니다.

예를 들어, 당신이 총 4개의 키워드에 해당하는 데이터를 준비하였다고 한다면, 각 폴더명을 keyword_1, keyword_2, keyword_3, keyword_4로 설정한 후 키워드에 해당하는 파일을 넣어줍니다.
위의 폴더 명들이 label로 자동으로 생성되는 것을 모델 훈련이 끝나면 확인할 수 있습니다.
그 다음 이 keyword 폴더들을 한 폴더안에 모아줍니다. 이 폴더 이름을 keyword라고 한다면
*--data_dir=keyword* 로 데이터 경로를 지정할 수 있습니다. 


## 모델 훈련

이제까지는 딥러닝에서 모두가 손 대기를 꺼려하는 작업이었습니다. 지금부터는 딥러닝에서 재미에 해당하는 부분입니다. 우리는 이제 speech_commands 예제에서 사용하는 모델의 아키텍쳐를 그대로 가져와서 그것을 우리만의 데이터셋에 적용할 것입니다. 다음의 커맨드를 실행하면 모델이 훈련될 것입니다.

In [3]:
!python3 tensorflow-lite/tensorflow/examples/speech_commands/train.py \
    --model_architecture=conv \
    --how_many_training_steps=10000,100  \
    --train_dir=./retrain_logs \
    --data_dir=data
    --wanted_words=bulyiya,suzy

/Library/Frameworks/Python.framework/Versions/3.7/Resources/Python.app/Contents/MacOS/Python: can't open file 'tensorflow-lite/tensorflow/examples/speech_commands/train.py': [Errno 2] No such file or directory


런타임이 모델을 훈련시키는 동안, 우리가 방금 실행시킨 커맨드가 어떤 작업을 하는지를 확인해 보겠습니다.

- ```python3 tensorflow-lite/tensorflow/examples/speech_commands/train.py``` 훈련을 실행시키는 스크립트 파일
- ```--model_architecture=conv``` 훈련할 모델 아키텍쳐. speech_commands 예제에서는 ```"conv"```라고 하는 모델 아키텍쳐를 사용하였습니다. 사용가능한 다른 모델 아키텍쳐들의 상세한 내용은 ```tensorflow-lite/tensorflow/examples/speech_commands/models.py```에서 확인할 수 있습니다.
- ```--how_many_training_steps=10000,100``` 훈련을 반복할 횟수. 10000번의 가중치 업데이트 후 더 작은 learning-rate를 사용하여 100번의 추가 업데이트가 있을 것입니다.
- ```--train_dir=./retrain_logs``` 훈련의 중간 결과 가중치들이 저장되는 경로. 이 결과파일들은 모델을 재훈련, 추론, 고정 및 tflite로 변환하는데 필요합니다.
- ```--data_dir=data``` 앞에서 16bit .wav파일로 변환한 파일을 담고있는 폴더들의 상위폴더. 이 폴더는 bulyiya와 suzy 디렉토리를 하위 디렉토리로 가지고 있어야 합니다.
- ```--wanted_words=bulyiya,suzy``` 인식하고자 하는 키워드들. ```--data_dir```에서 지정한 경로 안에 같은 이름으로 되어 훈련데이터를 가지고 있는 폴더가 각 키워드별로 있어야 합니다.

만약 훈련을 중단한 모델을 중간부터 다시 훈련시키고자 한다면 ```--start_checkpoint=./retrain_logs/conv.ckpt-1100```와 같이 특정 체크포인트를 지정해서 그곳에서부터 다시 훈련을 시작할 수 있습니다.

이 외에도 소스코드를 직접 수정하지 않고도 여러가지 조건을 변경하여 모델을 훈련시킬 수 있습니다. 이에 관한 상세한 내용은 ```tensorflow-lite/tensorflow/examples/speech_commands/train.py```의 FLAGS 입력 부분에서 확인할 수 있습니다.

## 모델 고정

오래 기다리셨습니다. 혹은 당신이 훌륭한 장비를 사용할 수 있는 축복받은 사람이라면 오래 기다리지 않았을 수도 있습니다. 이제 훈련의 결과로 생성된 체크포인트를 추론환경에서 사용하기 위하여 고정할 것입니다. 훈련 중간결과를 저장하기 위한 목적인 체크포인트 파일을 고정한 .pb파일은 체크포인트파일에서 훈련에 필요한 기능들을 제거하고 최적화하여 저장할 것입니다.

(주: 2.0 version부터는 saved_model이라고 하는 api를 지원합니다. 기존의 pb로 만들어진 API보다 한단계 더 추상화가 된 API입니다. 불행히도, pb모델과 saved_model간의 변환은 자유롭지 못합니다. 그러나 다행인 것은 pb모델을 tflite모델로 변환하는 방법과 saved_model을 tflite모델로 변환하는 방법에는 크게 차이가 없다는 것입니다. 따라서 본 예제의 내용흐름을 잘 따라올 수 있으면 2.x version의 텐서플로우에서도 큰 어려움 없이 tflite모델을 생성하는 파이프라인을 구성할 수 있을 것입니다.)

In [12]:
%cd /Users/sanghunkang/dev/2020-contributon/
!python3 tensorflow-lite/tensorflow/examples/speech_commands/freeze.py \
    --model_architecture=conv \
    --output_file=./content/retrain_logs/conv.pb \
    --start_checkpoint=./content/retrain_logs/conv.ckpt-10100 \
    --wanted_words=bulyiya,suzy

/Users/sanghunkang/dev/2020-contributon
2020-09-13 16:40:12.060728: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-09-13 16:40:12.083694: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbce90b07b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-13 16:40:12.083722: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
{
  "desired_samples": 16000,
  "window_size_samples": 480,
  "window_stride_samples": 160,
  "spectrogram_length": 98,
  "fingerprint_width": 40,
  "fingerprint_size": 3920,
  "label_count": 4,
  "sample_rate": 16000,
  "preprocess": "mfcc",
  "average_window_width": -1
}
DECODE>>> Tensor("decoded_sample_data_1d:0", shape=(16000,), dtype=float32) DecodeWav(audio=<tf.Tensor 'decoded_sample_data:0' shape=(16000, 1) dtype=float32>, sample_rate=<tf.Tensor 'decode

커맨드에서 사용한 FLAGS들의 세부 내용은 다음과 같습니다.
- ```!python3 tensorflow-lite/tensorflow/examples/speech_commands/freeze.py``` 고정을 실행하는 스크립트
- ```--model_architecture=conv``` 사용할 모델 아키텍쳐. 훈련에 사용한 것과 동일한 모델 아키텍쳐를 사용합니다.
- ```--output_file=./content/retrain_logs/conv.pb``` 고정한 결과 .pb파일이 저장될 경로
- ```--start_checkpoint=./content/retrain_logs/conv.ckpt-10100``` 고정시키고자 하는 중간훈련 버전. 체크포인트파일은 특별한 설정을 하지 않으면 일정한 단계마다 ```conv.ckpt-1000.data-00000-of-00001```, ```conv.ckpt-1000.index```, ```conv.ckpt-1000.meta``` 와 같은 파일들을 생성합니다. 훈련한 체크포인트 파일중 가장 퍼포먼스가 좋았던 버전을 지정하서 사용하면 될 것입니다.
- ```--wanted_words=bulyiya,suzy``` 인식하고자 하는 키워드들. 훈련에 사용한 것과 동일한 키워드들을 사용합니다.

## 그래프 탐색
파이썬 메모리상이 아닌 일반적인(?) 명칭으로서 우리가 훈련한 모델의 각 부분이 어떻게 되는지를 알아볼 필요가 있습니다. 그래프를 탐색해서 알아낸 내부구조에 특정 부분을 우리는 tflite모델을 만들면서 input과 output으로 모델 외부와 상호작용할 수 있는 채널로 개방할 것입니다. 이 형식은 이후 iOS앱 개발에서 반드시 지켜줘야 하는 형식이므로 우리는 그 형식을 정확하게 iOS개발자에게 전달하기 위해서 모델의 그래프구조가 freezing 상태에서 어떻게 생겼는지 확인할 필요가 있습니다. 

In [ ]:
filename = "content/retrain_logs/conv.pb"
with tf.io.gfile.GFile(filename, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')
with tf.compat.v1.Session() as sess:
    for n in tf.get_default_graph().as_graph_def().node:
        print(n.name, n.op, n.input)

출력결과는 텐서플로 그래프 상의 각 레이어의 이름, 레이어가 수행하는 연산, 레이어가 받는 입력들을 보여줍니다. 레이어가 받는 입력값이 iOS에서 입력으로 넣어주는 레이어일 것입니다. 이 부분은 우리의 예제에서는 ```decoded_sample_data```에 해당합니다. 따라서 .tflite 생성에 사용할 입력값 레이어는 ```--input_arrays=decoded_sample_data,decoded_sample_data:1```입니다. 마찬가지로 출력레이어로 사용할 레이어는 ```labels_softmax```입니다. 그러므로 .tflite 생성에 사용할 출력값 레이어는 ```--output_arrays=labels_softmax```입니다.

## .tflite 파일 생성

그래프가 정의된 .pb파일을 interpreter로 컴파일 합니다.

In [24]:
!toco --graph_def_file=./content/retrain_logs/conv.pb \
    --output_file=./content/retrain_logs/conv.tflite \
    --input_arrays=decoded_sample_data,decoded_sample_data:1 \
    --output_arrays=labels_softmax \
    --allow_custom_ops

2020-09-13 17:08:32.208072: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-09-13 17:08:32.274506: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbf52ec29b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-13 17:08:32.274545: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-13 17:08:32.308944: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2020-09-13 17:08:32.309157: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2020-09-13 17:08:32.352659: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:786] Optimization results for grappler item: graph_to_optimize
2020-09-13 17:08:32.352699: I tensorflow/core/grapple

## Inference

wav로 들어올 지 bufferStream으로 들어올지를 결정하는 것은 iOS 개발자의 결정사항이지만, 일단 input레이어에 약속된대로 데이터를 꽂고 나면 output레이어에서 모델의 추론 결과를 약속한대로 뱉어내야 합니다. 그것이 잘 되는지를 Python 코드 상에서 Interpreter와 예시 웨이브파일을 사용하여 확인할 것입니다. 우선 생성한 .tflite 파일이 그래프 구조, 그 중에서도 특히 입출력구조가 동일한지를 확인해봅시다.

In [23]:
import numpy as np
from scipy.io import wavfile
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="content/retrain_logs/conv.tflite")
print(interpreter.get_input_details())
for tensor_detail in interpreter.get_tensor_details():
    print(tensor_detail)

print(interpreter.get_input_details())
print(interpreter.get_output_details())

[{'name': 'decoded_sample_data', 'index': 12, 'shape': array([16000,     1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}, {'name': 'decoded_sample_data:1', 'index': 13, 'shape': array([], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0)}]
{'name': 'AudioSpectrogram', 'index': 0, 'shape': array([], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
{'name': 'Conv2D_1_bias', 'index': 1, 'shape': array([64], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
{'name': 'Conv2D_bias', 'index': 2, 'shape': array([64], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
{'name': 'MatMul_bias', 'index': 3, 'shape': array([4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
{'name': 'MaxPool2d', 'index': 4, 'shape': array([], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
{'name': 'Mfcc', 'index': 5, 'shape': array([], dtype=

In [22]:
# 인터프리터에 텐서들을 배정합니다. 이 작업을 통해서 데이터가 입력될 때 데이터가 계산되는 길이 정의됩니다.
interpreter.allocate_tensors()

# 인식하고자 하는 웨이브파일을 읽어옵니다.
samplerate, data = wavfile.read("data/bulyiya/불이야_test_1.wav")

# 첫 번쨰 input을 입력합니다. 그래프를 분석하면 (16000, 1)shape의 [0,1)의 np.float32임을 확인할 수 있습니다. 그것에 맞춰서 데이터를 수정합니다.
input_data = np.array(data[:16000]/32767.0, dtype=np.float32).reshape((16000, 1))
interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_data)
# 두 번째 input을 입력합니다. 16it 웨이브 파일이므로 16000이어야 합니다.
interpreter.set_tensor(interpreter.get_input_details()[1]['index'], np.int32(samplerate)) 
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

inference [[2.4671772e-07 1.6052921e-04 9.9778628e-01 2.0529309e-03]]


In [ ]:
이제 텐서들의 흐름 - 그러므로 텐서플로우(!) - 에 구체적인 데이터들이 채워졌습니다. 이제 이 텐서들의 흐름 끝에 우리가 최종적으로 얻고자 했던 인식결과가 나타나는지를 확인해보겠습니다.

In [ ]:
# 키워드 인식을 실제로 실행시킵니다.
interpreter.invoke()

# 실행결과를 읽어옵니다.
print("inference", interpreter.get_tensor(interpreter.get_output_details()[0]['index']))

결과는 길이가 4인 array입니다. 여기서 처음 두개의 값은 각각 silence와 unknown에 해당하는 값입니다. 마지막 두 개의 값 중 가장 높은 인식 결과가 입력한 파일의 분류와 같다면 커스텀 후 빌드한 모델은 훌륭하게 동작하고 있는 것입니다.

여기까지 진행되었으면 이제 iOS에 배포할 모델의 준비는 끝났습니다. 모델이 준비되었다는 기쁜 소식을 iOS개발자에게 전달하고, 우리는 퇴근할 준비를 하도록 합시다!